In [0]:
def mounting_sa(storage_account_name, container_name):
    #getting credetials from secret scope
    client_id = dbutils.secrets.get("Secret-scope-formula1","secret-formula1-client-id-service-principal")
    tenant_id =dbutils.secrets.get("Secret-scope-formula1","secret-tenant-id-service-principal-formula1")
    secret = dbutils.secrets.get("Secret-scope-formula1","secret-formula1-client-secret-service-principal")
    #setting up configs
    configs = {"fs.azure.account.auth.type": "OAuth",
        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="Secret-scope-formula1",key="secret-formula1-client-secret-service-principal"),
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    #unmount if already exists
    mount_point = f"/mnt/{storage_account_name}/{container_name}"
    if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(mount_point)
    #mouting all the folders
    dbutils.fs.mount(
      source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
      mount_point = f"/mnt/{storage_account_name}/{container_name}",
      extra_configs = configs)



In [0]:
mounting_sa("saformula1dbricks","raw")
mounting_sa("saformula1dbricks","processed")
mounting_sa("saformula1dbricks","reporting")

/mnt/saformula1dbricks/raw has been unmounted.
/mnt/saformula1dbricks/processed has been unmounted.
/mnt/saformula1dbricks/reporting has been unmounted.


In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/mnt/saformula1dbricks/reporting,abfss://reporting@saformula1dbricks.dfs.core.windows.net/,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/saformula1dbricks/demo,abfss://demo@saformula1dbricks.dfs.core.windows.net/,
/Volume,DbfsReserved,
/mnt/saformula1dbricks/raw,abfss://raw@saformula1dbricks.dfs.core.windows.net/,
/volumes,DbfsReserved,


In [0]:
reporting_race_dframe = spark.read.format("delta").load("/mnt/saformula1dbricks/processed/races/")
reporting_circuits_dframe = spark.read.format("delta").load("/mnt/saformula1dbricks/processed/circuits")

display(reporting_race_dframe)
display(reporting_circuits_dframe)

raceId,year,round,circuitId,name,ingestion_date,race_timestamp
1053,2021,2,21,Emilia Romagna Grand Prix,2024-12-03T23:52:00.705Z,2021-04-18 13:00:00
1052,2021,1,3,Bahrain Grand Prix,2024-12-03T23:52:00.705Z,2021-03-28 15:00:00
1051,2021,21,1,Australian Grand Prix,2024-12-03T23:52:00.705Z,2021-11-21 06:00:00
1054,2021,3,20,TBC,2024-12-03T23:52:00.705Z,2021-05-02 \N
1055,2021,4,4,Spanish Grand Prix,2024-12-03T23:52:00.705Z,2021-05-09 13:00:00
1056,2021,5,6,Monaco Grand Prix,2024-12-03T23:52:00.705Z,2021-05-23 13:00:00
1057,2021,6,73,Azerbaijan Grand Prix,2024-12-03T23:52:00.705Z,2021-06-06 12:00:00
1058,2021,7,7,Canadian Grand Prix,2024-12-03T23:52:00.705Z,2021-06-13 18:00:00
1059,2021,8,34,French Grand Prix,2024-12-03T23:52:00.705Z,2021-06-27 13:00:00
1060,2021,9,70,Austrian Grand Prix,2024-12-03T23:52:00.705Z,2021-07-04 13:00:00


circuitId,circuitRef,name,location,country,lat,lng,alt,url,timestamp,year,month,day
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit,2024-12-03T23:50:10.288Z,2024,12,3
2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.738,18,http://en.wikipedia.org/wiki/Sepang_International_Circuit,2024-12-03T23:50:10.288Z,2024,12,3
3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7,http://en.wikipedia.org/wiki/Bahrain_International_Circuit,2024-12-03T23:50:10.288Z,2024,12,3
4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya,2024-12-03T23:50:10.288Z,2024,12,3
5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.405,130,http://en.wikipedia.org/wiki/Istanbul_Park,2024-12-03T23:50:10.288Z,2024,12,3
6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,7,http://en.wikipedia.org/wiki/Circuit_de_Monaco,2024-12-03T23:50:10.288Z,2024,12,3
7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5,-73.5228,13,http://en.wikipedia.org/wiki/Circuit_Gilles_Villeneuve,2024-12-03T23:50:10.288Z,2024,12,3
8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.8642,3.16361,228,http://en.wikipedia.org/wiki/Circuit_de_Nevers_Magny-Cours,2024-12-03T23:50:10.288Z,2024,12,3
9,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694,153,http://en.wikipedia.org/wiki/Silverstone_Circuit,2024-12-03T23:50:10.288Z,2024,12,3
10,hockenheimring,Hockenheimring,Hockenheim,Germany,49.3278,8.56583,103,http://en.wikipedia.org/wiki/Hockenheimring,2024-12-03T23:50:10.288Z,2024,12,3


In [0]:
# Clean up the target path if it exists
dbutils.fs.rm("/mnt/saformula1dbricks/reporting", True)

# Write the DataFrame as a Delta table
circuits_by_country.write.format("delta").mode("overwrite").save("/mnt/saformula1dbricks/reporting")

In [0]:
# Inner join between races and circuits on circuitId
joined_df = reporting_race_dframe.join(reporting_circuits_dframe, 
                                       reporting_race_dframe.circuitId == reporting_circuits_dframe.circuitId, 
                                       "inner")

display(joined_df)


raceId,year,round,circuitId,name,ingestion_date,race_timestamp,circuitId,circuitRef,name,location,country,lat,lng,alt,url,timestamp,year,month,day
1053,2021,2,21,Emilia Romagna Grand Prix,2024-12-03T23:52:00.705Z,2021-04-18 13:00:00,21,imola,Autodromo Enzo e Dino Ferrari,Imola,Italy,44.3439,11.7167,37,http://en.wikipedia.org/wiki/Autodromo_Enzo_e_Dino_Ferrari,2024-12-03T23:50:10.288Z,2024,12,3
1052,2021,1,3,Bahrain Grand Prix,2024-12-03T23:52:00.705Z,2021-03-28 15:00:00,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7,http://en.wikipedia.org/wiki/Bahrain_International_Circuit,2024-12-03T23:50:10.288Z,2024,12,3
1051,2021,21,1,Australian Grand Prix,2024-12-03T23:52:00.705Z,2021-11-21 06:00:00,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit,2024-12-03T23:50:10.288Z,2024,12,3
1054,2021,3,20,TBC,2024-12-03T23:52:00.705Z,2021-05-02 \N,20,nurburgring,Nürburgring,Nürburg,Germany,50.3356,6.9475,578,http://en.wikipedia.org/wiki/N%C3%BCrburgring,2024-12-03T23:50:10.288Z,2024,12,3
1055,2021,4,4,Spanish Grand Prix,2024-12-03T23:52:00.705Z,2021-05-09 13:00:00,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya,2024-12-03T23:50:10.288Z,2024,12,3
1056,2021,5,6,Monaco Grand Prix,2024-12-03T23:52:00.705Z,2021-05-23 13:00:00,6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,7,http://en.wikipedia.org/wiki/Circuit_de_Monaco,2024-12-03T23:50:10.288Z,2024,12,3
1057,2021,6,73,Azerbaijan Grand Prix,2024-12-03T23:52:00.705Z,2021-06-06 12:00:00,73,BAK,Baku City Circuit,Baku,Azerbaijan,40.3725,49.8533,-7,http://en.wikipedia.org/wiki/Baku_City_Circuit,2024-12-03T23:50:10.288Z,2024,12,3
1058,2021,7,7,Canadian Grand Prix,2024-12-03T23:52:00.705Z,2021-06-13 18:00:00,7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5,-73.5228,13,http://en.wikipedia.org/wiki/Circuit_Gilles_Villeneuve,2024-12-03T23:50:10.288Z,2024,12,3
1059,2021,8,34,French Grand Prix,2024-12-03T23:52:00.705Z,2021-06-27 13:00:00,34,ricard,Circuit Paul Ricard,Le Castellet,France,43.2506,5.79167,432,http://en.wikipedia.org/wiki/Paul_Ricard_Circuit,2024-12-03T23:50:10.288Z,2024,12,3
1060,2021,9,70,Austrian Grand Prix,2024-12-03T23:52:00.705Z,2021-07-04 13:00:00,70,red_bull_ring,Red Bull Ring,Spielburg,Austria,47.2197,14.7647,678,http://en.wikipedia.org/wiki/Red_Bull_Ring,2024-12-03T23:50:10.288Z,2024,12,3


In [0]:
# Grouping by country and counting the number of races per country
country_race_count_df = joined_df.groupBy("country") \
                                 .count() \
                                 .orderBy("count", ascending=False)

# Display the total number of races per country
display(country_race_count_df)

country,count
Italy,102
Germany,80
UK,76
USA,71
Monaco,67
Belgium,66
France,62
Spain,58
Canada,51
Brazil,48
